Crawler fro the netherlands:

In [11]:
!pip install bs4
!pip install selenium

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
    --------------------------------------- 0.2/10.5 MB 4.0 MB/s eta 0:00:03
   - -------------------------------------- 0.4/10.5 MB 4.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/10.5 MB 4.4 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/10.5 MB 4.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.1/10.5 MB 4.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/10.5 MB 4.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/10.5 MB 4.9 MB/s eta 0:00:02
   ------- -------------------------------- 1.9/10.5 MB 5.0 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.5 MB 5.2 MB/s eta 0:00:02
   --------- ------------------------------ 2.4/10.5 MB 5.2 MB/s eta 0:00:02
   ---------- ----------------------------- 2.7/10.5 MB 5.3 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.5 MB 5.3 MB/s eta 0:00:02
   ---

In [32]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL of the company list page
base_url = "https://nwba.nl"

# URL of the company list page
company_list_url = "https://nwba.nl/nl/leden"

# Send a GET request to the company list URL and get the response
response = requests.get(company_list_url)

# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the <a> tags inside <div class="raxo-readmore">
company_links = soup.select('div.raxo-readmore a')

# Open a CSV file for writing
csv_file = open('data/netherlands_member.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.DictWriter(csv_file, fieldnames=['Company Name', 'Description', 'Address', 'City', 'Postal Code', 'Contact Person', 'Email'])
csv_writer.writeheader()

# Extract the company information
for link in company_links:
    # Get the URL of the company page
    company_url = base_url + link['href']

    # Send a GET request to the company URL and get the response
    company_response = requests.get(company_url)

    # Parse the HTML content of the company response using BeautifulSoup
    company_soup = BeautifulSoup(company_response.content, 'html.parser')

    # Extract company description
    company_description = company_soup.find('p', class_='content-description')
    if company_description:
        company_description = company_description.text.strip()
    else:
        company_description = None

    # Extract company address details
    address_fields = company_soup.select('ul.fields-container li')
    address_info = {field.find('span', class_='field-label').text.strip(): field.find('span', class_='field-value').text.strip() for field in address_fields}

    # Extract address, city, and postal code
    company_name = address_info.get('Bedrijfsnaam:','')
    address = address_info.get('Adres:', '')
    city = address_info.get('Plaats:', '')
    postal_code = address_info.get('Postcode:', '')

    # Extract contact person and email
    contact_person = address_info.get('Contactpersoon:', '')
    email = address_info.get('E-mail:', '')

    # Write the data to the CSV file
    csv_writer.writerow({
        'Company Name': company_name,
        'Description': company_description,
        'Address': address,
        'City': city,
        'Postal Code': postal_code,
        'Contact Person': contact_person,
        'Email': email
    })

# Close the CSV file
csv_file.close()

In [35]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL of the company list page
base_url = "https://nwba.nl"

# URL of the company list page
company_list_url = "https://nwba.nl/nl/leden"

# Send a GET request to the company list URL and get the response
response = requests.get(company_list_url)

# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the <a> tags inside <div class="raxo-readmore">
company_links = soup.select('div.raxo-readmore a')

# Open a CSV file for writing
csv_file = open('data/netherlands_member.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.DictWriter(csv_file, fieldnames=['Company Name', 'Description', 'Address', 'City', 'Postal Code', 'Contact Person', 'Email'])
csv_writer.writeheader()

# Extract the company information
for link in company_links:
    # Get the URL of the company page
    company_url = base_url + link['href']

    # Send a GET request to the company URL and get the response
    company_response = requests.get(company_url)

    # Parse the HTML content of the company response using BeautifulSoup
    company_soup = BeautifulSoup(company_response.content, 'html.parser')

    # Extract company description
    company_description = company_soup.find('p', class_='content-description')
    if company_description:
        company_description = company_description.text.strip()
    else:
        company_description = None

    # Extract company address details
    address_fields = company_soup.select('ul.fields-container li')
    address_info = {field.find('span', class_='field-label').text.strip(): field.find('span', class_='field-value').text.strip() for field in address_fields}

    # Extract address, city, and postal code
    company_name = address_info.get('Bedrijfsnaam:','')
    address = address_info.get('Adres:', '')
    city = address_info.get('Plaats:', '')
    postal_code = address_info.get('Postcode:', '')

    # Extract contact person and email
    contact_person = address_info.get('Contactpersoon:', '')
    email = address_info.get('E-mail:', '')

    # Write the data to the CSV file
    csv_writer.writerow({
        'Company Name': company_name,
        'Description': company_description,
        'Address': address,
        'City': city,
        'Postal Code': postal_code,
        'Contact Person': contact_person,
        'Email': email
    })

# Close the CSV file
csv_file.close()

Now working on a carlwer for poland: https://h2poland.eu/en/hydrogen-investment-map/

In [9]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to crawl
url = "https://h2poland.eu/en/hydrogen-investment-map/"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all div elements with class "col-12 col-md-6 col-lg-4 my-3"
div_elements = soup.find_all("div", class_="col-12 col-md-6 col-lg-4 my-3")

# Iterate through each div element and extract the text content from child elements
for div in div_elements:
    # Extract text content from child elements within the div
    text_content = [element.get_text(strip=True) for element in div.find_all(["h3", "p", "a"])]

    # Join the text content with a newline character
    combined_text = "\n".join(text_content)

    # Print the extracted text content
    print(combined_text)

In [27]:
import pandas as pd

# Read the Excel file
df = pd.read_excel(r'C:\Users\André\Desktop\Universität\Universität\Master\Masterarbeit\webcrawler\data\poland_members.xlsx')

# Combine address information into a single column
df['Full Address'] = df[['Street', 'ZIP']].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)

# Save the updated DataFrame to the Excel file
df.to_excel(r'C:\Users\André\Desktop\Universität\Universität\Master\Masterarbeit\webcrawler\data\poland_members.xlsx')


Crawler for UK Members: https://ukhea.co.uk/industry-showcase/#

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://ukhea.co.uk/industry-showcase/#"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "elementor-column")))

# Get the HTML source of the rendered page
html_source = driver.page_source

# Save the HTML source to a file
with open("ukhea_page.html", "w", encoding="utf-8") as html_file:
    html_file.write(html_source)

In [37]:
# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(html_source, "html.parser")

# Find all the company sections on the page
company_sections = soup.find_all("div", class_="jet-listing-grid__item")

# Loop through each company section to extract information
for company_section in company_sections:
    # Find the company name
    company_name = company_section.find("h2", class_="elementor-heading-title elementor-size-default").text.strip()

    # Find the company address
    company_address = company_section.find("span", class_="elementor-icon-list-text").text.strip()

    # Find all the services provided by the company
    services = [li.text.strip() for li in company_section.find_all("span", class_="elementor-icon-list-text")[1:]]

    # Print the extracted information
    print(f"Company Name: {company_name}")
    print(f"Address: {company_address}")
    print("Services:")
    for service in services:
        print(f"- {service}")
    print()

# Close the browser
driver.quit()

Company Name: 2G Energy
Address: Unit 1 Sycamore Court, Warrington Road, Runcorn, Cheshire, WA7 1RS
Services:

Company Name: Action Sealtite
Address: 
Services:

Company Name: Acute Civil Engineering
Address: 3 Manor Court, Salesbury Hall Road, Ribchester, PR3 3XR
Services:

Company Name: Air Products
Address: Air Products PLC, Hersham Technology Park, Molesey Road, Hersham, KT12 4RZ
Services:
- Hydrogen Production
- Fuel Handling
- Hydrogen Refuelling Infrastructure
- Stationary Applications

Company Name: Alcazar
Address: 7 Bell Yard, London, WC2A 2JR
Services:
- Sector Support Functions
- Studies

Company Name: Ames Goldsmith Ceimig
Address: Units 1-3 Smeaton Road, Wester Gourdie Industrial Estate, Dundee, Tayside, DD2 4UT
Services:

Company Name: Anglo American
Address: 17 Charterhouse St, London, EC1N 6RA
Services:
- Hydrogen Production
- Investment
- Transport Applications
- Studies
- Misc.

Company Name: Ash Group
Address: 
Services:

Company Name: ATEQ
Address: Unit 71, Washfor

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://ukhea.co.uk/industry-showcase/#"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "jet-alphabet-list")))

# Find all the letter elements
letter_elements = driver.find_elements(By.CSS_SELECTOR, "div.jet-alphabet-list__row label")

# Loop through each letter element and click it
for letter_element in letter_elements:
    # Create an ActionChains instance to perform the click action
    action = ActionChains(driver)
    action.move_to_element(letter_element).click().perform()

    # Wait for the page to update
    time.sleep(2)

    # Get the updated HTML source
    html_source = driver.page_source

    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(html_source, "html.parser")

    # Find all the company sections on the page
    company_sections = soup.find_all("div", class_="jet-listing-grid__item")

    # Loop through each company section to extract information
    for company_section in company_sections:
        # Find the company name
        company_name = company_section.find("h2", class_="elementor-heading-title elementor-size-default").text.strip()

        # Find the company address
        company_address = company_section.find("span", class_="elementor-icon-list-text").text.strip()

        # Find all the services provided by the company
        services = [li.text.strip() for li in company_section.find_all("span", class_="elementor-icon-list-text")[1:]]

        # Print the extracted information
        print(f"Company Name: {company_name}")
        print(f"Address: {company_address}")
        print("Services:")
        for service in services:
            print(f"- {service}")
        print()

# Close the browser
driver.quit()

AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv

# Set up the Selenium webdriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# URL of the page to crawl
url = "https://ukhea.co.uk/industry-showcase/#"

# Navigate to the URL
driver.get(url)

# Wait for the page to load completely
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "jet-alphabet-list")))

# Find all the letter elements
letter_elements = driver.find_elements(By.CSS_SELECTOR, "div.jet-alphabet-list__row label")

# Function to extract company information from the page
def extract_company_information(soup):
    company_sections = soup.find_all("div", class_="jet-listing-grid__item")
    company_data = []
    for company_section in company_sections:
        company_name = company_section.find("h2", class_="elementor-heading-title elementor-size-default").text.strip()
        company_address = company_section.find("span", class_="elementor-icon-list-text").text.strip().replace("\n", " ")
        services = [li.text.strip() for li in company_section.find_all("span", class_="elementor-icon-list-text")[1:]]
        company_data.append((company_name, company_address, services))
    return company_data

# Open a CSV file for writing
csv_file = open('data/uk_member.csv', mode='w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Company Name', 'Address', 'Services'])

# Loop through each letter element and click it
for letter_element in letter_elements:
    # Create an ActionChains instance to perform the click action
    action = ActionChains(driver)
    action.move_to_element(letter_element).click().perform()

    # Wait for the page to update
    time.sleep(2)

    # Get the updated HTML source
    html_source = driver.page_source

    soup = BeautifulSoup(html_source, "html.parser")
    company_data = extract_company_information(soup)

    # Handle cases where tuples have more than 3 values
    for data in company_data:
        values = list(data)
        company_name = values[0]
        company_address = values[1]
        services = ', '.join(sum([values[2], []], []))  # Flatten nested list
        csv_writer.writerow([company_name, company_address, services])

# Close the CSV file and the browser
csv_file.close()
driver.quit()